In [1]:
import pandas as pd

In [2]:
min_grant_perc = 0.30 # 30% of per capita entitlement

# Victoria

In [3]:
vic_councils = pd.read_excel("../Data/FA Grants Data Tables.xlsx", sheet_name="Victoria")

In [4]:
vic_councils.drop(columns = [column for column in vic_councils.columns if "Unnamed" in column], inplace=True)

In [5]:
total_grant_vic = vic_councils["Grant unadjusted"].sum()
current_per_capita_minimum_vic = 25.60
per_capita_vic = current_per_capita_minimum_vic / min_grant_perc

In [6]:
vic_councils["Deficit"] = vic_councils["Funding Gap"].apply(lambda x: 1 if x > 0 else 0)
vic_councils["Minimum Grant (PC Basis)"] = min_grant_perc * per_capita_vic * vic_councils["ERP"]

total_deficit_vic = vic_councils["Deficit"] @ vic_councils["Funding Gap"]

vic_councils["Raw Allocation"] = vic_councils["Funding Gap"] * vic_councils["Deficit"] * total_grant_vic / total_deficit_vic

vic



NameError: name 'vic' is not defined

# South Australia

In [50]:
import numpy as np
import scipy.optimize as opt

In [51]:
sa_councils = pd.read_excel("../Data/FA Grants Data Tables - Worked.xlsx", sheet_name="South Australia")

/home/spulick/miniforge3/envs/fagrants/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Web Extension extension is not supported and will be removed
  warn(msg)


In [52]:
sa_councils

,Council,Population (current year),Per Capita Minimum Council (PCM),Total Raw Calc,Per Capita Applied Amount,Per Capita Applied Amount (per head),Estimated Grant (GPG),Estimated Grant (per head),Deficit,Minimum Grant (PC basis),...,Adjustment,New Grant Adjusted,Change,New Grant Per Capita,Change %,Change from prev year,Per Capita Change,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,Adelaide,27901,PCM,-71481892,710359,25.46,710359,25.46,0,710359.46,...,-0.460000,710359,0.085328,25.460000,0,8.53,0.00,NaN,NaN,NaN
1,Adelaide Hills,41842,PCM,-4454563,1065297,25.46,1065297,25.46,0,1065297.32,...,-0.320000,1065297,0.025707,25.460000,0,2.57,0.00,NaN,NaN,NaN
2,Adelaide Plains,10932,-,5052887,1892230,173.09,1717083,157.07,1,278328.72,...,-171829.903539,1717083,0.100000,172.787496,0,10.00,-0.30,NaN,NaN,NaN
3,Alexandrina,30323,-,5057998,1894144,62.47,1844007,60.81,1,772023.58,...,-46816.540735,1844007,0.013649,62.356084,0,1.36,-0.11,NaN,Total Grant to State,1.531528e+08
4,Barossa,26268,-,6430573,2408153,91.68,1779006,67.73,1,668783.28,...,-624925.122317,1779006,0.150000,91.515575,0,15.00,-0.16,NaN,Per Capita,8.486667e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,West Torrens,64519,PCM,-20788774,1642654,25.46,1642654,25.46,0,1642653.74,...,0.260000,1642654,0.038816,25.460000,0,3.88,0.00,NaN,NaN,NaN
64,Whyalla,21902,-,15713243,5884374,268.67,5728616,261.56,1,557624.92,...,-145441.238792,5728616,0.090329,268.197299,0,9.03,-0.47,NaN,NaN,NaN
65,Wudinna,1161,-,4248089,1590845,1370.24,1548736,1333.97,1,29559.06,...,-39320.516499,1548736,0.017463,1367.835070,0,1.75,-2.40,NaN,NaN,NaN
66,Yankalilla,6100,-,289390,108372,17.77,227235,37.25,1,155306.00,...,71929.000000,227235,-0.050001,25.460000,0,-5.00,7.69,NaN,NaN,NaN


In [53]:
sa_non_min = sa_councils[sa_councils["MGC"] != 1]

In [54]:
total_grants = sa_non_min["New Grant"].sum()

In [55]:
G_t = np.array(sa_non_min["New Grant"])
G_t_1 = np.array(sa_non_min["2023 - 2024"])

In [71]:
def objective(x):
    return np.sum((G_t + x - G_t_1)/G_t_1)

x0 = np.repeat(G_t.mean(), len(G_t))

In [72]:
lower_bounds = (np.repeat(-0.15, len(G_t)) + 1 - G_t/G_t_1) * G_t_1
upper_bounds = (np.repeat(0.30, len(G_t)) + 1 - G_t/G_t_1) * G_t_1

In [58]:
linear_constraint = opt.LinearConstraint(np.identity(len(G_t)), lower_bounds, upper_bounds)
sum_constraint = opt.LinearConstraint(np.ones(len(G_t)), 0, 0)

In [59]:
res = opt.minimize(
    objective,
    x0,
    method='trust-constr',
    constraints=[linear_constraint, sum_constraint],
    options={'disp': True, 'maxiter': 100000}
)

/home/spulick/miniforge3/envs/fagrants/lib/python3.13/site-packages/scipy/optimize/_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


`xtol` termination condition is satisfied.
Number of iterations: 957, function evaluations: 66450, CG iterations: 3420, optimality: 1.21e-06, constraint violation: 0.00e+00, execution time: 1.1e+02 s.


In [60]:
res.x

array([-330553.35984869,  -71696.245665  , -856969.11996293,
         14998.63663182, -160353.55935536, -127082.55509596,
       -264814.29488502,  -20956.53174537,  -22716.283775  ,
        423372.9931766 , -679613.709707  ,  -52606.01828255,
        -21277.83932206,  -56149.50396939, -589285.64044006,
        121328.91024716,  361221.97027183,  303317.27923764,
          4779.05634356,  -22883.78537784, -965539.31313489,
       -110883.33206779, -176354.88404898, -325236.21439674,
         23761.20616517, -126577.16936951, -122720.27542576,
       -340561.35147857,  571109.79016224,   69658.63425438,
       2104106.4355014 ,  -29766.28307033,  -12627.52707718,
       1500131.38065968, -638218.70765507, -543815.18992881,
       -627234.07204859,  -99852.90870265, -265385.22620336,
       1922490.73652464,  110748.3802464 ,    6131.93549548,
        454471.31701005,   73230.74082133,  -98571.58200437,
        522841.37292138, -476358.24687792,  -66652.21913604,
       -284387.82561197]

In [66]:
x = res.x

In [67]:
objective(x)

np.float64(0.6654365496674431)

In [68]:
objective(sa_non_min["Adjustment"])

np.float64(2.1047620278928805)

In [69]:
k = (G_t + x - G_t_1)/G_t_1 * np.identity(len(G_t))

In [70]:
k.diagonal()

array([-1.68192283e-03, -2.73227562e-05,  1.52177663e-09, -5.40729895e-10,
       -1.74170092e-04, -1.97956399e-04,  1.85440029e-10, -4.01405969e-07,
        7.30663304e-11, -3.07803179e-03, -5.17646124e-04,  1.73610440e-10,
       -5.20639772e-06,  1.88903520e-10, -8.01674515e-05, -7.18509692e-03,
       -2.62945819e-04, -2.94356855e-05, -3.46827248e-03, -3.99156017e-10,
       -5.77007316e-08, -3.31245810e-10, -1.64116763e-02, -2.20990676e-02,
       -3.02619229e-03, -5.42705058e-03, -1.03098224e-02, -1.62875816e-02,
       -9.85692397e-03, -1.18010812e-03,  1.23638126e-01,  9.30188434e-11,
       -8.48532494e-03,  2.02451784e-01, -2.11106452e-02, -3.88335058e-03,
       -5.54124611e-03, -3.12674916e-03,  1.34867189e-09,  1.28808689e-01,
       -1.20786944e-09, -8.56027276e-03, -1.26954002e-02, -1.60823171e-09,
       -2.24577144e-04, -1.77648920e-02,  2.73459008e-02, -4.92527456e-04,
        7.85239539e-10])